In [1]:
from bs4 import BeautifulSoup
from requests_html import HTML
import pandas
import pathlib
import requests
import time
import uuid

film_links = list()
pages = [f'https://www.ozmovies.com.au/movies/browse?page={x}' for x in range(127)]
for p in pages:
    time.sleep(4)
    r = requests.get(p)
    r = r.text.split('<div class="movie-image pull-left">')[1:]
    for x in r:
        html = HTML(html=x.split('<div class="inline-image">')[0])
        for y in html.links:
            film_links.append(y)

print(len(film_links))

1061


In [2]:
dataframe = pandas.DataFrame(columns=['title', 'year', 'activity', 'agent'])
for x in film_links:
    time.sleep(4)
    r = requests.get(f'https://www.ozmovies.com.au/{x}')
    title = r.text.split('<h2>')[1].split('</h2>')[0]
    year = [k for k in r.text.split('<li class="active">')][1:]
    year = [k.split('</li>')[0] for k in year]
    year = [BeautifulSoup(b, 'lxml').get_text().replace('\n', '').strip() for b in year]
    year = [k for k in year if k in [str(z) for z in range(1880, 2030)]][0]
    r = r.text.split('<div class="movie-primary-details-people pull-left">')[1].split('<div class="pull-right">')[0]
    for a in r.split(' <div class="row">')[1:]:
        activity = a.split('<div class="pull-left personType">')[1].split('</div>')[0]
        agent = a.split('<div class="pull-left personName">')[1].split('<br />')
        agent = [BeautifulSoup(b, 'lxml').get_text().replace('\n', '').strip() for b in agent]
        for h in agent:
            dataframe.loc[len(dataframe)] = [(title), (year), (activity), (h)]

print(len(dataframe))
dataframe.head()

16323


,title,year,activity,agent
0,13 Gantry Row,1998,Key cast:,Rebecca Gibney
1,13 Gantry Row,1998,Key cast:,John Adam
2,13 Gantry Row,1998,Key cast:,Doris Younane
3,13 Gantry Row,1998,Key cast:,Nicholas Hammond
4,13 Gantry Row,1998,Key cast:,Mark Gerber


In [3]:
dataframe['label'] = dataframe['title']+' ('+dataframe['year'].astype('str')+')'
dataframe = dataframe.replace({'Key cast:':'hasActor', 'Director/s:':'hasDirector', 'Producer/s:':'hasProducer'})

dataframe['agent'] = dataframe['agent'].fillna('DROP')
dataframe = dataframe.loc[~dataframe.agent.isin(['DROP'])]

work_auth = dataframe.copy()
work_auth = work_auth[['label']].drop_duplicates()
work_auth['work_uuid'] = [str(uuid.uuid4()) for x in range(len(work_auth))]
dataframe = pandas.merge(dataframe, work_auth, on='label', how='left')

agent_auth = dataframe.copy()
agent_auth = agent_auth[['agent']].drop_duplicates()
agent_auth['agent_uuid'] = [str(uuid.uuid4()) for x in range(len(agent_auth))]
dataframe = pandas.merge(dataframe, agent_auth, on='agent', how='left')

dataframe.to_csv(pathlib.Path.cwd() / 'ozmovies.csv', index=False)
print(len(dataframe))
dataframe.head()

16323


,title,year,activity,agent,label,work_uuid,agent_uuid
0,13 Gantry Row,1998,hasActor,Rebecca Gibney,13 Gantry Row (1998),e5055786-5d3a-4217-99a8-612897e41483,dfb83346-039e-492b-8ff1-489c23cf5b5d
1,13 Gantry Row,1998,hasActor,John Adam,13 Gantry Row (1998),e5055786-5d3a-4217-99a8-612897e41483,9a6d8e9d-c029-4f06-868f-ee5236da4d56
2,13 Gantry Row,1998,hasActor,Doris Younane,13 Gantry Row (1998),e5055786-5d3a-4217-99a8-612897e41483,485457bf-fa7d-4dac-a60e-274d117707d4
3,13 Gantry Row,1998,hasActor,Nicholas Hammond,13 Gantry Row (1998),e5055786-5d3a-4217-99a8-612897e41483,71785510-f456-4d8c-8900-99f08a2950a8
4,13 Gantry Row,1998,hasActor,Mark Gerber,13 Gantry Row (1998),e5055786-5d3a-4217-99a8-612897e41483,66b7c78b-3496-4264-92bc-c1523718933f
